In [4]:
def core(k):

import cv2
import numpy as np
import cPickle
import time
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import cluster


start = time.time()

# read the train and test files

train_images_filenames, train_labels, test_images_filenames, test_labels = read_data('train_images_filenames.dat', 'train_labels.dat', 'test_images_filenames.dat', 'test_labels.dat')

# create the SIFT detector object

detector = cv2.SIFT(nfeatures=100)

# read the just 30 train images per class
# extract SIFT keypoints and descriptors
# store descriptors in a python list of numpy arrays
Train_descriptors, Train_label_per_descriptor = extract_train_features(train_images_filenames, train_labels, detector)

#as default k=512
codebook = compute_codebook("codebook.dat", Train_descriptors, k= 50)


visual_words = BoW(codebook, Train_descriptors)

# Train a linear SVM classifier

clf, stdSlr = train_SVM(visual_words, train_labels)

accuracy = evaluate_test(test_images_filenames, test_labels, codebook, stdSlr)
print 'Final accuracy: ' + str(accuracy)

end=time.time()
print 'Done in '+str(end-start)+' secs.'

return
## 49.56% in 285 secs.

NameError: name 'read_data' is not defined

In [ ]:
def read_data(train_files, train_labels, test_files, test_labels):
    
    train_images_filenames = cPickle.load(open(train_files,'r'))
    test_images_filenames = cPickle.load(open(test_files,'r'))
    train_labels = cPickle.load(open(train_labels,'r'))
    test_labels = cPickle.load(open(test_labels,'r'))

    print 'Loaded '+str(len(train_images_filenames))+' training images filenames with classes ',set(train_labels)
    print 'Loaded '+str(len(test_images_filenames))+' testing images filenames with classes ',set(test_labels)
    
    return (train_images_filenames, train_labels, test_images_filenames, test_labels)


In [ ]:
def extract_train_features(train_images_filenames, train_labels, detector):
        
    Train_descriptors = []
    Train_label_per_descriptor = []

    for i in range(len(train_images_filenames)):
        
        kpt,des= compute_feature(train_images_filenames[i], detector)
        
        Train_descriptors.append(des)
        Train_label_per_descriptor.append(train_labels[i])
        print str(len(kpt))+' extracted keypoints and descriptors'


    return (Train_descriptors, Train_label_per_descriptor)


In [ ]:
def compute_codebook(codebook_name, Train_descriptors, k=512):

    # Transform everything to numpy arrays
    size_descriptors=Train_descriptors[0].shape[1]
    D=np.zeros((np.sum([len(p) for p in Train_descriptors]),size_descriptors),dtype=np.uint8)
    startingpoint=0
    for i in range(len(Train_descriptors)):
        D[startingpoint:startingpoint+len(Train_descriptors[i])]=Train_descriptors[i]
        startingpoint+=len(Train_descriptors[i])
        
    print 'Computing kmeans with '+str(k)+' centroids'
    init=time.time()
    codebook = cluster.MiniBatchKMeans(n_clusters=k, verbose=False, batch_size=k * 20,compute_labels=False,reassignment_ratio=10**-4)
    codebook.fit(D)
    cPickle.dump(codebook, open(codebook_name, "wb"))
    end=time.time()
    print 'Done in '+str(end-init)+' secs.'
    return codebook

In [ ]:
def BoW(codebook, descriptors):
    #cluster_centers_ : array, [n_clusters, n_features]
    init=time.time()
    #visual_words=np.zeros((len(descriptors),k),dtype=np.float32)
    k = codebook.cluster_centers_.shape[0]
    print k
    visual_words=np.zeros((len(descriptors),k),dtype=np.float32)
    for i in xrange(len(descriptors)):
        words=codebook.predict(descriptors[i])
        visual_words[i,:]=np.bincount(words,minlength=k)

    end=time.time()
    print 'Done in '+str(end-init)+' secs.'
    
    return visual_words


In [ ]:
def train_SVM(visual_words, train_labels, toStore= False, filename = ""):

    stdSlr = StandardScaler().fit(visual_words)
    D_scaled = stdSlr.transform(visual_words)
    print 'Training the SVM classifier...'
    clf = svm.SVC(kernel='linear', C=1).fit(D_scaled, train_labels)
    print 'Done!'
    
    if toStore:
        cPickle.dump(clf, open(filename, "wb"))
    return clf, stdSlr

In [ ]:
def compute_feature(image_filename, detector):

    print 'Reading image '+ image_filename
    ima=cv2.imread(image_filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    kpt,des=detector.detectAndCompute(gray,None)
    
    return kpt,des
        


In [ ]:
def evaluate_test(test_images_filenames, test_labels, codebook, stdSlr):
    k = codebook.cluster_centers_.shape[0]
    # get all the test data and predict their labels
    visual_words_test=np.zeros((len(test_images_filenames), k),dtype=np.float32)
    for i in range(len(test_images_filenames)):
        #extract features for a single image
        kpt,des= compute_feature(test_images_filenames[i], detector)
        #extract VW for a single image
        words=codebook.predict(des)
        visual_words_test[i,:]=np.bincount(words,minlength=k)
        
    return 100*clf.score(stdSlr.transform(visual_words_test), test_labels)

In [ ]:
#Import libraries
import cv2
import numpy as np
import cPickle
import time
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import cluster


cbook_size_k = np.arange(64,2048,64)

for i in range(len(cbook_size_k)):
    print 'i='+str(i)+'.'
    print cbook_size_k[i]
    core(cbook_size_k[i])
    

print 'Overall finished'